1. 방어적프로그래밍
2. 관심사의분리,개발지침약어
3. 컴포지션과상속
4. 함수와 메서드의인자

# 1. How arguments are copied to functions

- immutable이 argument로 넘어오는 경우
  1. local variable로 copy하고,
  2. 아래의 과정을 수행함
```python
new_argument = argument + <expression>
argument = new_argument
```

- mutalbe이 넘어오는 경우
  1. reference만 copy $\Rightarrow$ 여전히 동일한 객체를 바라보고 있음  
  (참고: https://stackoverflow.com/a/23525870)


In [ ]:
def my_function(argument):
  argument += " in function"
  print(argument)

In [ ]:
immutable = 'hello'

In [ ]:
my_function(immutable)

hello in function


In [ ]:
immutable

'hello'

In [ ]:
mutable = list('hello')

In [ ]:
my_function(mutable)

['h', 'e', 'l', 'l', 'o', ' ', 'i', 'n', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n']


In [ ]:
mutable

['h',
 'e',
 'l',
 'l',
 'o',
 ' ',
 'i',
 'n',
 ' ',
 'f',
 'u',
 'n',
 'c',
 't',
 'i',
 'o',
 'n']

In [8]:
import dis

In [ ]:
dis.dis(my_function)

  2           0 LOAD_FAST                0 (argument)
              2 LOAD_CONST               1 (' in function')
              4 INPLACE_ADD
              6 STORE_FAST               0 (argument)

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                0 (argument)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [ ]:
def my_function_v2(argument):

  if hasattr(argument, 'copy'):
    argument = argument.copy()
  else:
    argument = argument
  
  argument += " in function"
  print(argument)

In [ ]:
dis.dis(my_function_v2)

  3           0 LOAD_GLOBAL              0 (hasattr)
              2 LOAD_FAST                0 (argument)
              4 LOAD_CONST               1 ('copy')
              6 CALL_FUNCTION            2
              8 POP_JUMP_IF_FALSE       20

  4          10 LOAD_FAST                0 (argument)
             12 LOAD_METHOD              1 (copy)
             14 CALL_METHOD              0
             16 STORE_FAST               0 (argument)
             18 JUMP_FORWARD             4 (to 24)

  6     >>   20 LOAD_FAST                0 (argument)
             22 STORE_FAST               0 (argument)

  8     >>   24 LOAD_FAST                0 (argument)
             26 LOAD_CONST               2 (' in function')
             28 INPLACE_ADD
             30 STORE_FAST               0 (argument)

  9          32 LOAD_GLOBAL              2 (print)
             34 LOAD_FAST                0 (argument)
             36 CALL_FUNCTION            1
             38 POP_TOP
             40 LOAD_CO

In [5]:
def my_function_v3(argument):

  # only work for str type argument
  argument = argument + " in function"
  print(argument)

In [9]:
dis.dis(my_function_v3)

  3           0 LOAD_FAST                0 (argument)
              2 LOAD_CONST               1 (' in function')
              4 BINARY_ADD
              6 STORE_FAST               0 (argument)

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                0 (argument)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


# 2. Variable number of arguments
## 2.1. Unpacking Positional Arguments
- Pythonic style
```python
inputs = [1, 2]
# un-Pythonic
multi_arg(inputs[0], inputs[1])
a, b = inputs[0], inputs[1]
# Pythonic
multi_arg(*inputs)
a, b = inputs
```

In [ ]:
def multi_arg(arg1, arg2):
  print(f'Argument 1: {arg1}')
  print(f'Argument 2: {arg2}')

In [ ]:
inputs = [1, 2]

In [ ]:
multi_arg(inputs[0], inputs[1])

Argument 1: 1
Argument 2: 2


In [ ]:
multi_arg(*inputs)

Argument 1: 1
Argument 2: 2


- partial unpacking

In [ ]:
inputs = [1, 2, 3, 4, 5]

In [ ]:
first, *rest, last = inputs

In [ ]:
print(f'First: {first}')
print(f'Rest: {rest}')
print(f'Last: {last}')

First: 1
Rest: [2, 3, 4]
Last: 5


In [ ]:
class User:
    def __init__(self, user_id, first_name, last_name):
        self.user_id = user_id
        self.first_name = first_name
        self.last_name = last_name

    def __repr__(self):
        return f"{self.__class__.__name__}({self.user_id!r}, {self.first_name!r}, {self.last_name!r})"


def bad_users_from_rows(dbrows) -> list:
    """A bad case (non-pythonic) of creating ``User``s from DB rows."""
    return [User(row[0], row[1], row[2]) for row in dbrows]


def users_from_rows(dbrows) -> list:
    """Create ``User``s from DB rows."""
    return [
        User(user_id, first_name, last_name)
        for (user_id, first_name, last_name) in dbrows
    ]


In [ ]:
USERS = [(i, f"first_name_{i}", f"last_name_{i}") for i in range(1_000)]

In [ ]:
 bad_users_from_rows(USERS[:10])

[User(0, 'first_name_0', 'last_name_0'),
 User(1, 'first_name_1', 'last_name_1'),
 User(2, 'first_name_2', 'last_name_2'),
 User(3, 'first_name_3', 'last_name_3'),
 User(4, 'first_name_4', 'last_name_4'),
 User(5, 'first_name_5', 'last_name_5'),
 User(6, 'first_name_6', 'last_name_6'),
 User(7, 'first_name_7', 'last_name_7'),
 User(8, 'first_name_8', 'last_name_8'),
 User(9, 'first_name_9', 'last_name_9')]

In [ ]:
users_from_rows(USERS[:10])

[User(0, 'first_name_0', 'last_name_0'),
 User(1, 'first_name_1', 'last_name_1'),
 User(2, 'first_name_2', 'last_name_2'),
 User(3, 'first_name_3', 'last_name_3'),
 User(4, 'first_name_4', 'last_name_4'),
 User(5, 'first_name_5', 'last_name_5'),
 User(6, 'first_name_6', 'last_name_6'),
 User(7, 'first_name_7', 'last_name_7'),
 User(8, 'first_name_8', 'last_name_8'),
 User(9, 'first_name_9', 'last_name_9')]

## 2.2. Unpacking Keyword Arguments
- kwargs는 dictionary로 packing됨

In [ ]:
def multi_kwarg(**kwargs):
  print(kwargs)

In [ ]:
multi_kwarg(key='value')

{'key': 'value'}


In [ ]:
multi_kwarg(key1='value1', key2='value2')

{'key1': 'value1', 'key2': 'value2'}


In [ ]:
inputs = {
    'key1': 'value1',
    'key2': 'value2'
}

In [ ]:
multi_kwarg(**inputs)

{'key1': 'value1', 'key2': 'value2'}


# 2.3. The number of arguments in functions
- function arguments가 너무 많을때의 대안으로 아래가 있음
  1. reificiation
    - argument들을 묶어서 object로 만들어서 해당 object를 입력으로 받도록 변경?
    - 일종의 abstraction
  2. positional/keyword argument를 입력으로 받는 dynamic signature로 변경

- 그러나 이 경우 유지보수 문제 등 복잡성이 늘어나기 때문에 더 작은 함수 단위로 쪼개는게 나을 수 있음  
$\Rightarrow$ 함수는 하나의 기능만 하도록 작성할것!

In [ ]:
def many_inputs_function(a, b, c, d):
  for x in [a,b,c,d]:
    print(x)

In [ ]:
many_inputs_function(1,2,3,4)

1
2
3
4


In [ ]:
def many_inputs_function_v2(*args, **kwargs):
  for x in args:
    print(x)
  
  for k, v in kwargs.items():
    print(k, v)

In [ ]:
many_inputs_function_v2(1,2,3,4,5)

1
2
3
4
5


In [ ]:
many_inputs_function_v2(1,2,3,4,5, a=1, b=2, c=3)

1
2
3
4
5
a 1
b 2
c 3


# 2.4. Function arguments and coupling

- 함수가 많은 인자를 입력으로 받을수록, 그 함수를 호출하는 다른함수와 tightly coupled 됨
- `f1, f2` 두개의 함수에 대해 아래를 가정하면
  - f1이 f2를 호출
  - f2는 5개의 arguments를 받음
- f2의 arguments들이 많아질수록 f1에서 변경하거나 처리해야될 코드가 늘어남
- f1이 f2에 대한 모든 정보를 가지고 있다면 f2는 leaky abstraction(?)
  - abstraction이 충분히 잘 된게 아니며, f1에 대해서만 유용하며 다른 코드에서 재사용하기 힘들다고 함

  ```python
  # tightly copupled 예시
  class Book:
      def __init__(self, title, subtitle, author):
          self.title = title
          self.subtitle = subtitle
          self.author = author
  def display_book_info(book):
      print(f'{book.title}: {book.subtitle} by {book.author}')
  # cohesion을 통해 coupling 을 줄임
  class Book:
      def __init__(self, title, subtitle, author):
          self.title = title
          self.subtitle = subtitle
          self.author = author
    def display_info(self):
        print(f'{self.title}: {self.subtitle} by {self.author}')
  ```

### [참고 자료]
- https://freecontent.manning.com/achieving-loose-coupling/
- http://rapapa.net/?p=3266
- https://www.joelonsoftware.com/2002/11/11/the-law-of-leaky-abstractions/


# 2.5. Compact function signatures that take too many arguments

1. common object에 포함되면 object 자체를 넘기는게 좋음

  ```python
  track_request(request.headers, request.ip_addr, request.request_id)
  ```
  $$\Downarrow$$

  ```python
  track_request(request):
    request.headers
    request.ip_addr
    request.request_id
  ```

  단, 다른 object를 추가로 입력받는 경우 예상치 못한 side-effect들이 발생할수 있음


2. 비슷한 arguments들을 묶어서 grouping
  - 위 예제는 grouping이 이미 되어 있는 경우
  - reify를 통한 추상화

3. `*args, **kwargs`로 변경
  - signature, meaning, legibility는 대부분 잃게 되는 단점은 있음

# 3. Final remarks on good practices for software design

# 4. Orthogonality in software
- calling a method on an object **should not alter** the internal state of other (unrealted) objects.


In [ ]:
def calculate_price(base_price: float, tax: float, discount: float) -> float:
    """
    >>> calculate_price(10, 0.2, 0.5)
    6.0

    >>> calculate_price(10, 0.2, 0)
    12.0
    """
    return (base_price * (1 + tax)) * (1 - discount)


def show_price(price: float) -> str:
    """
    >>> show_price(1000)
    '$ 1,000.00'

    >>> show_price(1_250.75)
    '$ 1,250.75'
    """
    return "$ {0:,.2f}".format(price)


def str_final_price(
    base_price: float, tax: float, discount: float, fmt_function=str
) -> str:
    """

    >>> str_final_price(10, 0.2, 0.5)
    '6.0'

    >>> str_final_price(1000, 0.2, 0)
    '1200.0'

    >>> str_final_price(1000, 0.2, 0.1, fmt_function=show_price)
    '$ 1,080.00'

    """
    return fmt_function(calculate_price(base_price, tax, discount))


In [ ]:
str_final_price(1000, 0.2, 0.1)

'1080.0'

In [ ]:
# show_price 함수를 넘기지 않아도 str_final_price는 문제 없이 동작함
# show_price 함수의 변경이 calculate_price에 아무런 영향을 미치지 않음
str_final_price(1000, 0.2, 0.1, fmt_function=show_price)

'$ 1,080.00'

In [1]:
class BaseTokenizer:
    """
    >>> tk = BaseTokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
    >>> list(tk)
    ['28a2320b', 'fd3f', '4627', '9792', 'a2b38e3c46b0']
    """

    def __init__(self, str_token):
        self.str_token = str_token

    def __iter__(self):
        yield from self.str_token.split("-")


class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())


class Tokenizer(UpperIterableMixin, BaseTokenizer):
    """
    >>> tk = Tokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
    >>> list(tk)
    ['28A2320B', 'FD3F', '4627', '9792', 'A2B38E3C46B0']
    """


In [2]:
tk = Tokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")

In [4]:
list(tk)

['28A2320B', 'FD3F', '4627', '9792', 'A2B38E3C46B0']

# 5. Structuring the code
- packaging 관한 내용으로 특별한 건 없음
- 한 module내에 constants, functions, classes 등 너무 많은 내용을 정의하기보단 적절히 비슷한 기능끼리 묶어서 module을 여러개 작성할것을 권장
- python에선 `__init__.py`를 사용해서 directory를 구분하고 packaging하는게 가능
